In [312]:
import torch
import torch.nn as nn
import pandas as pd
from model import LSTM

In [313]:
# csv로 만든 파일 불러오기.
df = pd.read_csv('encoded.csv', index_col=0) # 0번째 열을 인덱스로 지정.
vocaDF = pd.read_csv('vocab.csv', index_col=0)
df.shape, vocaDF.shape

((760, 1696), (7954, 1))

In [314]:
vocaDF.columns

Index(['0'], dtype='object')

In [315]:
model_path = 'model.pth'

HIDDEN_SIZE = 64
EMBEDD_DIM  = 128
VOCAB_SIZE  = vocaDF.shape[0]
NUM_LAYERS = 1
EPOCHS      = 100
LR          = 0.1
BATCH_SIZE  = 1
DROPOUT     = 0.5
OUTPUT      = 8

mdl = LSTM(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT, OUTPUT)

c:\ProgramData\anaconda3\envs\NLP_38\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [316]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [317]:
def predict(model, text):
    with torch.no_grad():
        text = torch.tensor((text), dtype=torch.int64).to(device)
        text = text.unsqueeze(0)
        offsets = torch.tensor([0]).to(device)
        predicted_label = model(text)
        return predicted_label.argmax(1).item()
    
# def predict(model, text, text_pipeline):
#     with torch.no_grad():
#         text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(device)
#         text = text.unsqueeze(0)
#         offsets = torch.tensor([0]).to(device)
#         predicted_label = model(text, offsets)
#         return predicted_label.argmax(1).item() + 1

In [318]:
#text = '아름다운 청춘의 한 장 함께 써내려 가자 너와의 추억들로 가득 채울래 (come on!) 아무 걱정도 하지는 마, 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게.'
#text = '월요일엔 아마 바쁘지 않을까 화요일도 성급해 보이지 안 그래 수요일은 뭔가 어정쩡한 느낌 목요일은 그냥 내가 왠지 싫어'
#text = '다 너의 반 반 반의 반의 반도 채워주질 못 하네 채워지지가 않네 Yeah 딱 너의 반 반 반의 반이라도 내게 남았더라면 이렇게 붕 떠있진 않을 텐데'
text = '모두 할 말을 잃지 Like you 4차원 이상의 기적의 View 달콤히 찍어 문 빛의 퐁듀 보이기 시작한 음의 색도 예민해진 걸 느껴 뚜렷한 색감과 여섯 번째 감각'

In [319]:
def tokentext(text, vocaDF) :

    from konlpy.tag import Kkma
    kkma = Kkma()
    texttoken = kkma.morphs(text)

    # 어휘사전
    vocab_dict = {word: idx for idx, word in enumerate(vocaDF['0'].to_list())}
    encoded = [vocab_dict[token] for token in texttoken if token in vocab_dict]
    # 인코딩 결과 출력
    print("토큰화된 단어들:", texttoken)
    print("인코딩 결과:", encoded)
    padded_id=[]
    max_length = df.shape[1]

    if len(encoded) < max_length :
        padded_id.append(encoded + [0]*(max_length-len(encoded)))
    else :
        padded_id.append(encoded)
    return padded_id[0]



In [320]:
token = tokentext(text, vocaDF)

토큰화된 단어들: ['모두', '하', 'ㄹ', '말', '을', '잃', '지', 'Like', 'you', '4', '차원', '이상', '의', '기적', '의', 'View', '닿', 'ㄹ', '콤히', '찍', '어', '문', '빛', '의', '퐁듀', '보이', '기', '시작', '하', 'ㄴ', '음의', '색도', '예민', '해지', 'ㄴ', '것', '을', '느끼', '어', '뚜렷', '하', 'ㄴ', '색감', '과', '여섯', '번째', '감각']
인코딩 결과: [3135, 984, 53, 263, 4808, 93, 94, 732, 1090]


In [321]:
result = predict(mdl, token) #[0: 'ballad', 1:'dance', 2: 'fork', 3:'hiphop', 4:'indi', 5:'R&B', 6:'rock', 7:'trot']

In [322]:
if result == 0: print('ballad')
elif result == 1 : print('dance')
elif result == 2 : print('fork')
elif result == 3 : print('hiphop')
elif result == 4 : print('indi')
elif result == 5 : print('R&B')
elif result == 6 : print('rock')
elif result == 7 : print('trot')

dance
